<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 3.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

In [4]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [5]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [6]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag
from oppdeling_trening_testsett import enzymfordeling

In [7]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [8]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

# Importering av relevant data

In [9]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_normal.index = pd.to_datetime(data_normal.index,
                                   format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [10]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29049 entries, 2022-10-31 17:37:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29049 non-null  int64  
 1   EnzymeType_A2  29049 non-null  int64  
 2   EnzymeType_B   29049 non-null  int64  
 3   EnzymeType_C   29049 non-null  int64  
 4   EnzymeType_D   29049 non-null  int64  
 5   EnzymeType_E   29049 non-null  int64  
 6   RawMatPercent  29049 non-null  float64
 7   NIRfat         29049 non-null  float64
 8   NIRash         29049 non-null  float64
 9   NIRwater       29049 non-null  float64
 10  TT08           29049 non-null  float64
 11  TT20           29049 non-null  float64
 12  TT12           29049 non-null  float64
 13  Mw             460 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [12]:
data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18542 entries, 2022-11-02 00:00:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18542 non-null  int64  
 1   EnzymeType_A2  18542 non-null  int64  
 2   EnzymeType_B   18542 non-null  int64  
 3   EnzymeType_C   18542 non-null  int64  
 4   EnzymeType_D   18542 non-null  int64  
 5   EnzymeType_E   18542 non-null  int64  
 6   RawMatPercent  18542 non-null  float64
 7   NIRfat         18542 non-null  float64
 8   NIRash         18542 non-null  float64
 9   NIRwater       18542 non-null  float64
 10  TT08           18542 non-null  float64
 11  TT20           18542 non-null  float64
 12  TT12           18542 non-null  float64
 13  Mw             124 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [13]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10507 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10507 non-null  int64  
 1   EnzymeType_A2  10507 non-null  int64  
 2   EnzymeType_B   10507 non-null  int64  
 3   EnzymeType_C   10507 non-null  int64  
 4   EnzymeType_D   10507 non-null  int64  
 5   EnzymeType_E   10507 non-null  int64  
 6   RawMatPercent  10507 non-null  float64
 7   NIRfat         10507 non-null  float64
 8   NIRash         10507 non-null  float64
 9   NIRwater       10507 non-null  float64
 10  TT08           10507 non-null  float64
 11  TT20           10507 non-null  float64
 12  TT12           10507 non-null  float64
 13  Mw             336 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [14]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering
resultater_kolonner = ["Treningsandel",
                       "RMSE test",
                       "R2 test",
                       "MAE test",
                       "MAPE test",
                       "RMSE trening",
                       "R2 trening",
                       "MAE trening",
                       "MAPE trening"]

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [15]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
treningssett_markert, enzymtyper_treningssett_markert, testsett,\
emzymtyper_testsett, treningssett_umarkert, _, treningssett\
=trening_testsett_oppdeling_enzym(rå_data,
                                  data,
                                  test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_trening_markert = enzymfordeling(rå_data,
                                           treningssett_markert)
enzymdata_test = enzymfordeling(rå_data,
                                testsett)


print("Enzymtyper Fordeling: Trening markert")
print(enzymdata_trening_markert)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett i forklaringsvariabler og respons
X_test = testsett.iloc[:, :-1]
y_test = testsett.iloc[:, -1]

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       128  34.78
E         60  16.30
B         56  15.22
C         56  15.22
D         50  13.59
A1        18   4.89
Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        32  34.78
E         15  16.30
B         14  15.22
C         14  15.22
D         13  14.13
A1         4   4.35


### RandomForestRegressor

In [16]:
# Modelloptimalisering
rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               gjentagelser = gjentagelser,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed = random_seed)

# Lagring av resultater
rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:14:20,340] A new study created in memory with name: no-name-302b790a-1bc1-4843-98d2-1b2161a9dbae
[I 2024-03-21 13:14:22,899] Trial 0 finished with value: 1025.3827070388936 and parameters: {'n_estimators': 183, 'max_depth': 3}. Best is trial 0 with value: 1025.3827070388936.
[I 2024-03-21 13:14:24,383] Trial 1 finished with value: 823.6728044762536 and parameters: {'n_estimators': 108, 'max_depth': 4}. Best is trial 1 with value: 823.6728044762536.


{'n_estimators': 108, 'max_depth': 4}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,767.779761,0.895372,549.635263,0.077084,661.236684,0.924122,482.305122,0.068853


### KNeighborsRegressor

In [17]:
# Modelloptimalisering
knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt=splitt,
                                               gjentagelser=gjentagelser,
                                               n_runder=n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


# Lagring av resultater
knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:14:25,077] A new study created in memory with name: no-name-48a86547-7326-42cd-b446-e23139bd5b64
[I 2024-03-21 13:14:25,220] Trial 0 finished with value: 644.2587830513249 and parameters: {'n_neighbors': 5, 'p': 7}. Best is trial 0 with value: 644.2587830513249.
[I 2024-03-21 13:14:25,384] Trial 1 finished with value: 636.1188613197064 and parameters: {'n_neighbors': 4, 'p': 10}. Best is trial 1 with value: 636.1188613197064.


{'n_neighbors': 4, 'p': 10}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,556.810246,0.944971,408.799193,0.060189,452.714184,0.964433,304.032683,0.046423


### SupportVectorRegressor

In [18]:
# Modelloptimalisering
svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt=splitt,
                                               gjentagelser=gjentagelser,
                                               n_runder=n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


# Lagring av resultater
svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:14:25,541] A new study created in memory with name: no-name-47ca6f7e-a313-4f26-8f10-614ca6773df4
[I 2024-03-21 13:14:25,649] Trial 0 finished with value: 695.7916577371055 and parameters: {'kernel': 'linear', 'C': 97, 'degree': 16, 'gamma': 'scale'}. Best is trial 0 with value: 695.7916577371055.
[I 2024-03-21 13:14:25,752] Trial 1 finished with value: 2089.4559056203 and parameters: {'kernel': 'poly', 'C': 75, 'degree': 8, 'gamma': 'auto'}. Best is trial 0 with value: 695.7916577371055.


{'kernel': 'linear', 'C': 97, 'degree': 16, 'gamma': 'scale'}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,607.960337,0.934397,446.722042,0.068245,627.751231,0.931612,437.395093,0.065203


## Alternativ 1: Flere treningsandeler

In [19]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data,
                                              test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i testsett
enzymdata_test = enzymfordeling(rå_data,
                                treningssett_markert)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.1, 0.3, 0.5, 0.7, 0.9]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2       128  34.78
E         60  16.30
B         56  15.22
C         56  15.22
D         50  13.59
A1        18   4.89


### RandomForestRegressor

In [20]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:14:26,060] A new study created in memory with name: no-name-01f4f957-1b27-4a87-9fd1-a8ae95c68187


Enzymtyper Fordeling: Treningssett markert andel: 0.1
    Frekvens  Andel
A2        11  34.38
B          5  15.62
C          5  15.62
E          5  15.62
D          4  12.50
A1         2   6.25


[I 2024-03-21 13:14:27,683] Trial 0 finished with value: 1570.9023871231143 and parameters: {'n_estimators': 151, 'max_depth': 1}. Best is trial 0 with value: 1570.9023871231143.
[I 2024-03-21 13:14:29,596] Trial 1 finished with value: 1288.162937874097 and parameters: {'n_estimators': 165, 'max_depth': 20}. Best is trial 1 with value: 1288.162937874097.
[I 2024-03-21 13:14:30,221] A new study created in memory with name: no-name-71528949-67e5-4772-83f3-ef8249af384d


Enzymtyper Fordeling: Treningssett markert andel: 0.3
    Frekvens  Andel
A2        33  34.38
E         16  16.67
B         15  15.62
C         15  15.62
D         13  13.54
A1         4   4.17


[I 2024-03-21 13:14:32,960] Trial 0 finished with value: 1105.8515364299049 and parameters: {'n_estimators': 137, 'max_depth': 5}. Best is trial 0 with value: 1105.8515364299049.
[I 2024-03-21 13:14:35,795] Trial 1 finished with value: 1105.5769962910022 and parameters: {'n_estimators': 127, 'max_depth': 5}. Best is trial 1 with value: 1105.5769962910022.
[I 2024-03-21 13:14:36,226] A new study created in memory with name: no-name-f65ed636-7b54-492d-b73a-98a0caa47d98


...

[I 2024-03-21 13:14:40,855] Trial 0 finished with value: 714.6888172752832 and parameters: {'n_estimators': 110, 'max_depth': 12}. Best is trial 0 with value: 714.6888172752832.
[I 2024-03-21 13:14:41,747] Trial 1 finished with value: 709.7517175580352 and parameters: {'n_estimators': 127, 'max_depth': 16}. Best is trial 1 with value: 709.7517175580352.
[I 2024-03-21 13:14:42,085] A new study created in memory with name: no-name-11158e49-e06f-48eb-8843-e407a07dd9fd


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
A2       101  34.95
E         48  16.61
B         44  15.22
C         44  15.22
D         39  13.49
A1        13   4.50


[I 2024-03-21 13:14:43,167] Trial 0 finished with value: 682.914785403458 and parameters: {'n_estimators': 141, 'max_depth': 13}. Best is trial 0 with value: 682.914785403458.
[I 2024-03-21 13:14:45,104] Trial 1 finished with value: 686.4268696371873 and parameters: {'n_estimators': 175, 'max_depth': 18}. Best is trial 0 with value: 682.914785403458.


{'trenings_andel': [0.1, 0.3, 0.5, 0.7, 0.9], 'parametere': [{'n_estimators': 165, 'max_depth': 20}, {'n_estimators': 127, 'max_depth': 5}, {'n_estimators': 192, 'max_depth': 17}, {'n_estimators': 127, 'max_depth': 16}, {'n_estimators': 141, 'max_depth': 13}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.1,1323.249420,0.682900,1011.654244,0.151222,454.414994,0.961164,339.678850,0.048644
1,0.3,1030.059052,0.807851,713.156624,0.102990,492.223537,0.961069,373.275917,0.054098
2,0.5,705.499234,0.909862,476.389542,0.071048,265.329943,0.988399,185.472711,0.027846
3,0.7,664.338848,0.920073,436.146779,0.066630,224.157095,0.991406,161.407733,0.024856
4,0.9,602.579835,0.934243,407.621460,0.061023,207.970698,0.992535,146.847301,0.022153


### KNeighborRegressor

In [21]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:14:46,383] A new study created in memory with name: no-name-2fad756e-108e-4559-8edb-5790b39da58a
[I 2024-03-21 13:14:46,761] Trial 0 finished with value: 2186.50578881214 and parameters: {'n_neighbors': 10, 'p': 5}. Best is trial 0 with value: 2186.50578881214.
[I 2024-03-21 13:14:46,954] Trial 1 finished with value: 2041.776579891975 and parameters: {'n_neighbors': 7, 'p': 1}. Best is trial 1 with value: 2041.776579891975.
[I 2024-03-21 13:14:47,161] A new study created in memory with name: no-name-52994dfb-e650-42c7-b039-6e0e620da8e2
[I 2024-03-21 13:14:47,405] Trial 0 finished with value: 1224.7076928597357 and parameters: {'n_neighbors': 9, 'p': 1}. Best is trial 0 with value: 1224.7076928597357.
[I 2024-03-21 13:14:47,575] Trial 1 finished with value: 1105.4563779951084 and parameters: {'n_neighbors': 6, 'p': 9}. Best is trial 1 with value: 1105.4563779951084.
[I 2024-03-21 13:14:47,847] A new study created in memory with name: no-name-e6c18625-d6cc-478e-98e5-68a9

{'trenings_andel': [0.1, 0.3, 0.5, 0.7, 0.9], 'parametere': [{'n_neighbors': 7, 'p': 1}, {'n_neighbors': 6, 'p': 9}, {'n_neighbors': 1, 'p': 8}, {'n_neighbors': 5, 'p': 10}, {'n_neighbors': 2, 'p': 3}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.1,1320.159416,0.684379,1047.484656,0.147460,1402.827512,0.629886,1105.099908,0.142033
1,0.3,913.332740,0.848932,667.560893,0.099258,761.574246,0.906805,573.744180,0.084996
2,0.5,738.298817,0.901286,544.620747,0.083083,0.000000,1.000000,0.000000,0.000000
3,0.7,723.714210,0.905148,516.598514,0.077646,528.899863,0.952154,377.241771,0.056599
4,0.9,657.749882,0.921651,458.067018,0.069831,337.122916,0.980385,226.146072,0.034090


### SupportVectorRegressor

In [22]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:14:49,530] A new study created in memory with name: no-name-08c9e06e-1b25-48d1-a3c7-dbede2c19422
[I 2024-03-21 13:14:49,587] Trial 0 finished with value: 2418.036879323521 and parameters: {'kernel': 'poly', 'C': 49, 'degree': 19, 'gamma': 'scale'}. Best is trial 0 with value: 2418.036879323521.
[I 2024-03-21 13:14:49,668] Trial 1 finished with value: 1450.8548063840935 and parameters: {'kernel': 'linear', 'C': 58, 'degree': 3, 'gamma': 'auto'}. Best is trial 1 with value: 1450.8548063840935.
[I 2024-03-21 13:14:49,736] A new study created in memory with name: no-name-95ace104-f66a-4cba-a497-0cdbe5e46dfa
[I 2024-03-21 13:14:49,809] Trial 0 finished with value: 42216.87387386595 and parameters: {'kernel': 'poly', 'C': 34, 'degree': 17, 'gamma': 'scale'}. Best is trial 0 with value: 42216.87387386595.
[I 2024-03-21 13:14:49,905] Trial 1 finished with value: 940.7742195855584 and parameters: {'kernel': 'linear', 'C': 84, 'degree': 10, 'gamma': 'scale'}. Best is trial 1 wit

{'trenings_andel': [0.1, 0.3, 0.5, 0.7, 0.9], 'parametere': [{'kernel': 'linear', 'C': 58, 'degree': 3, 'gamma': 'auto'}, {'kernel': 'linear', 'C': 84, 'degree': 10, 'gamma': 'scale'}, {'kernel': 'linear', 'C': 69, 'degree': 7, 'gamma': 'auto'}, {'kernel': 'linear', 'C': 47, 'degree': 12, 'gamma': 'auto'}, {'kernel': 'linear', 'C': 4, 'degree': 15, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.1,1150.109667,0.760452,909.265873,0.146192,1009.897622,0.808186,768.575321,0.119161
1,0.3,705.128003,0.909957,510.920556,0.080347,697.873860,0.921744,485.525084,0.072527
2,0.5,683.460751,0.915406,497.125489,0.078160,675.090593,0.924901,471.838805,0.069948
3,0.7,680.455436,0.916148,488.156421,0.076682,645.466492,0.928740,469.283812,0.070103
4,0.9,1203.618651,0.737644,990.461030,0.154307,1186.194648,0.757164,993.039147,0.149419


## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [23]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
trening_markert, test, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_trening_markert = enzymfordeling(rå_data,
                                           trening_markert)
enzymdata_test = enzymfordeling(rå_data,
                                test)


print("Enzymtyper Fordeling: Trening markert")
print(enzymdata_trening_markert)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)



# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       134  36.31
E         60  16.26
C         55  14.91
B         54  14.63
D         50  13.55
A1        16   4.34
Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        26  28.57
B         16  17.58
C         15  16.48
E         15  16.48
D         13  14.29
A1         6   6.59


### RandomForestregressor

In [24]:
# Modelloptimalisering
rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

print(parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:14:50,871] A new study created in memory with name: no-name-03d05fbb-854c-457b-928f-f1e546c7a07f
[I 2024-03-21 13:14:53,684] Trial 0 finished with value: 682.7600749432229 and parameters: {'n_estimators': 167, 'max_depth': 18}. Best is trial 0 with value: 682.7600749432229.
[I 2024-03-21 13:14:55,975] Trial 1 finished with value: 682.1892405170145 and parameters: {'n_estimators': 120, 'max_depth': 15}. Best is trial 1 with value: 682.1892405170145.


{'n_estimators': 120, 'max_depth': 15}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,707.672759,0.902668,493.914622,0.067242,215.681486,0.992045,144.9647,0.02219


### KNeighborsRegressor

In [25]:
# Modelloptimalisering
knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

print(parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:14:57,009] A new study created in memory with name: no-name-b23cabf4-a1f3-4f70-a2f1-033912025376
[I 2024-03-21 13:15:00,135] Trial 0 finished with value: 772.1859869370127 and parameters: {'n_neighbors': 2, 'p': 16}. Best is trial 0 with value: 772.1859869370127.
[I 2024-03-21 13:15:02,875] Trial 1 finished with value: 788.1765952921304 and parameters: {'n_neighbors': 7, 'p': 2}. Best is trial 0 with value: 772.1859869370127.


{'n_neighbors': 2, 'p': 16}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,696.147802,0.905813,499.037409,0.070699,327.840116,0.981621,222.331417,0.033658


### SupoortVectorRegressor

In [26]:
# Modelloptimalisering
svr_resultater, parametere = trening_svr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

print(parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:15:02,995] A new study created in memory with name: no-name-86ad2429-3f84-4def-b31f-116fa619b662
[I 2024-03-21 13:15:05,370] Trial 0 finished with value: 3674.2045694083467 and parameters: {'kernel': 'poly', 'C': 15, 'degree': 17, 'gamma': 'scale'}. Best is trial 0 with value: 3674.2045694083467.
[I 2024-03-21 13:15:07,109] Trial 1 finished with value: 1477.777759136396 and parameters: {'kernel': 'rbf', 'C': 52, 'degree': 20, 'gamma': 'scale'}. Best is trial 1 with value: 1477.777759136396.


{'kernel': 'rbf', 'C': 52, 'degree': 20, 'gamma': 'scale'}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1555.462752,0.529773,1335.057103,0.172,1457.567905,0.6367,1114.342294,0.152567


## Alternativ 2: Flere treningssandeler

In [28]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        26  28.57
B         16  17.58
C         15  16.48
E         15  16.48
D         13  14.29
A1         6   6.59


### RandomForestRegressor

In [29]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:16:51,200] A new study created in memory with name: no-name-c6382ccd-d198-426e-a5e2-92b7caf9b6e1


Enzymtyper Fordeling: Treningssett markert andel: 0.8
    Frekvens  Andel
A2       112  37.84
E         48  16.22
C         43  14.53
B         42  14.19
D         39  13.18
A1        12   4.05


[I 2024-03-21 13:16:53,643] Trial 0 finished with value: 685.089252559314 and parameters: {'n_estimators': 159, 'max_depth': 14}. Best is trial 0 with value: 685.089252559314.
[I 2024-03-21 13:16:57,747] Trial 1 finished with value: 1679.8869480327198 and parameters: {'n_estimators': 159, 'max_depth': 1}. Best is trial 0 with value: 685.089252559314.
[I 2024-03-21 13:17:00,238] A new study created in memory with name: no-name-68297d4a-4b4a-4e7b-811d-9d06af73fe7f


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
A2       125  37.31
E         54  16.12
B         48  14.33
C         48  14.33
D         46  13.73
A1        14   4.18


[I 2024-03-21 13:17:03,995] Trial 0 finished with value: 1084.0923797094226 and parameters: {'n_estimators': 169, 'max_depth': 3}. Best is trial 0 with value: 1084.0923797094226.
[I 2024-03-21 13:17:06,397] Trial 1 finished with value: 684.1183948374849 and parameters: {'n_estimators': 153, 'max_depth': 14}. Best is trial 1 with value: 684.1183948374849.
[I 2024-03-21 13:17:06,889] A new study created in memory with name: no-name-e250bab6-7850-4e0c-ac43-15e356af3a5e


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
A2       134  36.31
E         60  16.26
C         55  14.91
B         54  14.63
D         50  13.55
A1        16   4.34


[I 2024-03-21 13:17:09,175] Trial 0 finished with value: 682.6542141150491 and parameters: {'n_estimators': 122, 'max_depth': 14}. Best is trial 0 with value: 682.6542141150491.
[I 2024-03-21 13:17:12,911] Trial 1 finished with value: 685.1273100194086 and parameters: {'n_estimators': 185, 'max_depth': 9}. Best is trial 0 with value: 682.6542141150491.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_estimators': 159, 'max_depth': 14}, {'n_estimators': 153, 'max_depth': 14}, {'n_estimators': 122, 'max_depth': 14}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,719.991904,0.899250,508.084755,0.069340,213.157216,0.992230,144.091494,0.022088
1,0.9,717.034417,0.900076,505.729906,0.068977,212.344803,0.992289,144.038288,0.022082
2,1.0,721.434521,0.898846,503.231677,0.068770,214.702115,0.992117,145.353453,0.022231


### KNeighborRegressor

In [30]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:17:14,018] A new study created in memory with name: no-name-0d43dad9-5dc1-4511-9be0-354716916387
[I 2024-03-21 13:17:15,812] Trial 0 finished with value: 866.4647844181017 and parameters: {'n_neighbors': 17, 'p': 11}. Best is trial 0 with value: 866.4647844181017.
[I 2024-03-21 13:17:17,602] Trial 1 finished with value: 788.6763661643762 and parameters: {'n_neighbors': 7, 'p': 10}. Best is trial 1 with value: 788.6763661643762.
[I 2024-03-21 13:17:17,919] A new study created in memory with name: no-name-5c7165bf-2250-4423-ad17-07e4ef6f2bcf
[I 2024-03-21 13:17:19,714] Trial 0 finished with value: 799.0345006742443 and parameters: {'n_neighbors': 6, 'p': 18}. Best is trial 0 with value: 799.0345006742443.
[I 2024-03-21 13:17:21,525] Trial 1 finished with value: 740.7169641939672 and parameters: {'n_neighbors': 3, 'p': 4}. Best is trial 1 with value: 740.7169641939672.
[I 2024-03-21 13:17:21,579] A new study created in memory with name: no-name-75c59e21-7490-42b0-8acf-764

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_neighbors': 7, 'p': 10}, {'n_neighbors': 3, 'p': 4}, {'n_neighbors': 7, 'p': 19}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,752.908800,0.889827,537.197173,0.072614,503.636285,0.956625,346.736468,0.052543
1,0.9,701.825539,0.904270,491.312827,0.069256,400.157821,0.972618,268.088312,0.040008
2,1.0,762.787282,0.886917,542.120702,0.073196,506.049040,0.956208,348.132010,0.052714


### SupportVectorRegressor

In [31]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:17:26,219] A new study created in memory with name: no-name-3154e128-c4a8-4ee8-9949-68a402e07dce
[I 2024-03-21 13:17:28,449] Trial 0 finished with value: 1456.7980183617688 and parameters: {'kernel': 'rbf', 'C': 54, 'degree': 9, 'gamma': 'auto'}. Best is trial 0 with value: 1456.7980183617688.
[I 2024-03-21 13:17:30,184] Trial 1 finished with value: 1893.5305377361838 and parameters: {'kernel': 'rbf', 'C': 20, 'degree': 19, 'gamma': 'scale'}. Best is trial 0 with value: 1456.7980183617688.
[I 2024-03-21 13:17:30,475] A new study created in memory with name: no-name-1f18e407-1bc6-4be3-a1e1-e41d68a2f1f8
[I 2024-03-21 13:17:32,220] Trial 0 finished with value: 1226.5769741085387 and parameters: {'kernel': 'rbf', 'C': 78, 'degree': 9, 'gamma': 'auto'}. Best is trial 0 with value: 1226.5769741085387.
[I 2024-03-21 13:17:33,921] Trial 1 finished with value: 771.587316022096 and parameters: {'kernel': 'linear', 'C': 29, 'degree': 18, 'gamma': 'auto'}. Best is trial 1 with val

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'kernel': 'rbf', 'C': 54, 'degree': 9, 'gamma': 'auto'}, {'kernel': 'linear', 'C': 29, 'degree': 18, 'gamma': 'auto'}, {'kernel': 'rbf', 'C': 95, 'degree': 13, 'gamma': 'scale'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,1519.765487,0.551108,1305.669522,0.168779,1422.144749,0.654144,1085.836590,0.149456
1,0.9,671.795551,0.912287,501.026874,0.068064,657.488664,0.926076,464.123763,0.070524
2,1.0,1164.283346,0.736545,978.576917,0.127347,1045.802117,0.812972,769.841056,0.109197


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [32]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_design,
                                              test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.06
C         14  20.59
B         13  19.12
D         12  17.65
A2        10  14.71
A1         4   5.88


### RandomForestRegressor

In [33]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)

  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:18:27,776] A new study created in memory with name: no-name-be49a66f-fba8-44e8-b1bd-ee9327f51fc2


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
E         54  22.41
C         48  19.92
B         47  19.50
D         43  17.84
A2        37  15.35
A1        12   4.98


[I 2024-03-21 13:18:30,300] Trial 0 finished with value: 647.5656097762848 and parameters: {'n_estimators': 191, 'max_depth': 6}. Best is trial 0 with value: 647.5656097762848.
[I 2024-03-21 13:18:31,704] Trial 1 finished with value: 640.9695767871694 and parameters: {'n_estimators': 122, 'max_depth': 12}. Best is trial 1 with value: 640.9695767871694.
[I 2024-03-21 13:18:32,023] A new study created in memory with name: no-name-8e1feb49-5eda-464f-b0f1-8868974183dc


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
E         60  22.39
C         53  19.78
B         52  19.40
D         48  17.91
A2        41  15.30
A1        14   5.22


[I 2024-03-21 13:18:32,952] Trial 0 finished with value: 574.4469606128082 and parameters: {'n_estimators': 127, 'max_depth': 9}. Best is trial 0 with value: 574.4469606128082.
[I 2024-03-21 13:18:33,960] Trial 1 finished with value: 601.7942847962322 and parameters: {'n_estimators': 152, 'max_depth': 6}. Best is trial 0 with value: 574.4469606128082.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'n_estimators': 122, 'max_depth': 12}, {'n_estimators': 127, 'max_depth': 9}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,570.057596,0.932253,394.006346,0.050251,198.50758,0.992365,134.897373,0.018241
1,1.0,579.654873,0.929952,396.633687,0.051397,221.07372,0.990674,152.665592,0.020406


### KNeighborRegressor

In [34]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:18:34,351] A new study created in memory with name: no-name-f1c6a5b1-4101-48f7-9775-c3101cf64322
[I 2024-03-21 13:18:34,394] Trial 0 finished with value: 751.9651646181318 and parameters: {'n_neighbors': 9, 'p': 2}. Best is trial 0 with value: 751.9651646181318.
[I 2024-03-21 13:18:34,451] Trial 1 finished with value: 614.6076882204022 and parameters: {'n_neighbors': 4, 'p': 4}. Best is trial 1 with value: 614.6076882204022.
[I 2024-03-21 13:18:34,483] A new study created in memory with name: no-name-96bc5962-21b5-4103-adc6-0ddb752170c7
[I 2024-03-21 13:18:34,558] Trial 0 finished with value: 779.1236948489538 and parameters: {'n_neighbors': 9, 'p': 9}. Best is trial 0 with value: 779.1236948489538.
[I 2024-03-21 13:18:34,629] Trial 1 finished with value: 732.2713514510251 and parameters: {'n_neighbors': 8, 'p': 8}. Best is trial 1 with value: 732.2713514510251.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'n_neighbors': 4, 'p': 4}, {'n_neighbors': 8, 'p': 8}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,528.426081,0.941786,388.778176,0.049929,433.845246,0.963529,297.685393,0.040015
1,1.0,570.846252,0.932065,430.102778,0.056206,491.775424,0.953850,349.670548,0.047679


### SupportVectorRegressor

In [35]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:18:34,734] A new study created in memory with name: no-name-f2702e01-45ed-4dcf-bc1c-fe4093954ee0
[I 2024-03-21 13:18:34,789] Trial 0 finished with value: 627.9761743369187 and parameters: {'kernel': 'linear', 'C': 74, 'degree': 11, 'gamma': 'auto'}. Best is trial 0 with value: 627.9761743369187.
[I 2024-03-21 13:18:34,843] Trial 1 finished with value: 2166.9782849311687 and parameters: {'kernel': 'rbf', 'C': 16, 'degree': 16, 'gamma': 'auto'}. Best is trial 0 with value: 627.9761743369187.
[I 2024-03-21 13:18:34,869] A new study created in memory with name: no-name-adfdfa5b-216f-4d6d-a274-2584dbfc2dd3
[I 2024-03-21 13:18:34,935] Trial 0 finished with value: 2872.1980448419695 and parameters: {'kernel': 'poly', 'C': 98, 'degree': 19, 'gamma': 'scale'}. Best is trial 0 with value: 2872.1980448419695.
[I 2024-03-21 13:18:34,989] Trial 1 finished with value: 1904.5733279883082 and parameters: {'kernel': 'poly', 'C': 77, 'degree': 8, 'gamma': 'auto'}. Best is trial 1 with v

{'trenings_andel': [0.9, 1.0], 'parametere': [{'kernel': 'linear', 'C': 74, 'degree': 11, 'gamma': 'auto'}, {'kernel': 'poly', 'C': 77, 'degree': 8, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,587.490378,0.928046,426.758323,0.058071,580.484356,0.934709,390.263615,0.052453
1,1.0,1875.534877,0.266660,1523.475916,0.237049,1747.100153,0.417526,1325.424018,0.205951


## Alternativ 2: Flere treningssandeler

In [39]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.39
B         13  19.40
C         13  19.40
D         12  17.91
A2        10  14.93
A1         4   5.97


### RandomForestRegressor

In [40]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)

  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:20:19,767] A new study created in memory with name: no-name-a5302e03-639d-4dfc-9fb7-96a51a16958c


Enzymtyper Fordeling: Treningssett markert andel: 0.8
    Frekvens  Andel
E         48  22.22
C         43  19.91
B         42  19.44
D         38  17.59
A2        33  15.28
A1        12   5.56


[I 2024-03-21 13:20:21,776] Trial 0 finished with value: 995.9345065484648 and parameters: {'n_estimators': 101, 'max_depth': 12}. Best is trial 0 with value: 995.9345065484648.
[I 2024-03-21 13:20:23,760] Trial 1 finished with value: 1003.6656436643967 and parameters: {'n_estimators': 101, 'max_depth': 8}. Best is trial 0 with value: 995.9345065484648.
[I 2024-03-21 13:20:24,417] A new study created in memory with name: no-name-7d621cd7-6df6-4428-a9b2-2e07025d4376


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
E         54  22.22
C         48  19.75
B         47  19.34
D         44  18.11
A2        37  15.23
A1        13   5.35


[I 2024-03-21 13:20:27,815] Trial 0 finished with value: 997.0616049696887 and parameters: {'n_estimators': 139, 'max_depth': 8}. Best is trial 0 with value: 997.0616049696887.
[I 2024-03-21 13:20:33,134] Trial 1 finished with value: 986.5490704095589 and parameters: {'n_estimators': 116, 'max_depth': 6}. Best is trial 1 with value: 986.5490704095589.
[I 2024-03-21 13:20:33,400] A new study created in memory with name: no-name-754199b8-314a-4b16-bc7a-89aaf98a9297


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
E         60  22.30
C         54  20.07
B         52  19.33
D         48  17.84
A2        41  15.24
A1        14   5.20


[I 2024-03-21 13:20:35,878] Trial 0 finished with value: 987.2755899754584 and parameters: {'n_estimators': 137, 'max_depth': 19}. Best is trial 0 with value: 987.2755899754584.
[I 2024-03-21 13:20:38,348] Trial 1 finished with value: 1105.1271400605344 and parameters: {'n_estimators': 128, 'max_depth': 4}. Best is trial 0 with value: 987.2755899754584.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_estimators': 101, 'max_depth': 12}, {'n_estimators': 116, 'max_depth': 6}, {'n_estimators': 137, 'max_depth': 19}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,742.894553,0.869818,550.241142,0.069589,209.628696,0.991816,134.736135,0.017985
1,0.9,805.053814,0.847121,553.175213,0.068558,379.430451,0.973189,252.890072,0.035477
2,1.0,745.464147,0.868916,551.108868,0.069554,203.881916,0.992259,129.071182,0.017272


### KNeighborRegressor

In [41]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:20:39,180] A new study created in memory with name: no-name-9d0da329-bcf0-4694-83c4-48725005136a
[I 2024-03-21 13:20:42,064] Trial 0 finished with value: 1063.2648181773116 and parameters: {'n_neighbors': 15, 'p': 10}. Best is trial 0 with value: 1063.2648181773116.
[I 2024-03-21 13:20:45,319] Trial 1 finished with value: 995.9516809416609 and parameters: {'n_neighbors': 8, 'p': 12}. Best is trial 1 with value: 995.9516809416609.
[I 2024-03-21 13:20:45,548] A new study created in memory with name: no-name-752ddeea-8e81-4e69-92dd-1c3c4afc88ad
[I 2024-03-21 13:20:47,596] Trial 0 finished with value: 844.2055237673228 and parameters: {'n_neighbors': 6, 'p': 1}. Best is trial 0 with value: 844.2055237673228.
[I 2024-03-21 13:20:49,068] Trial 1 finished with value: 1043.1189031740287 and parameters: {'n_neighbors': 17, 'p': 4}. Best is trial 0 with value: 844.2055237673228.
[I 2024-03-21 13:20:49,094] A new study created in memory with name: no-name-7cf2f56a-e730-419e-a55f-

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_neighbors': 8, 'p': 12}, {'n_neighbors': 6, 'p': 1}, {'n_neighbors': 5, 'p': 6}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,662.362947,0.896512,502.480868,0.065794,499.016404,0.953626,349.165160,0.046215
1,0.9,631.795560,0.905843,441.899032,0.057413,431.206499,0.965373,294.585531,0.038896
2,1.0,633.123020,0.905447,464.611485,0.062872,449.987143,0.962291,309.416123,0.040799


### SupportVectorRegressor

In [42]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:20:53,248] A new study created in memory with name: no-name-d77016e5-1d17-4ec1-98f0-c080cea38475
[I 2024-03-21 13:20:55,662] Trial 0 finished with value: 1581.5242775404815 and parameters: {'kernel': 'rbf', 'C': 63, 'degree': 14, 'gamma': 'auto'}. Best is trial 0 with value: 1581.5242775404815.
[I 2024-03-21 13:20:58,724] Trial 1 finished with value: 1425.8433806024605 and parameters: {'kernel': 'rbf', 'C': 83, 'degree': 9, 'gamma': 'scale'}. Best is trial 1 with value: 1425.8433806024605.
[I 2024-03-21 13:20:59,173] A new study created in memory with name: no-name-1c8e2b8a-b39d-4b5d-93f0-60fea1e10aa6
[I 2024-03-21 13:21:02,914] Trial 0 finished with value: 3498.014740083453 and parameters: {'kernel': 'poly', 'C': 100, 'degree': 20, 'gamma': 'auto'}. Best is trial 0 with value: 3498.014740083453.
[I 2024-03-21 13:21:04,868] Trial 1 finished with value: 1976.751850501985 and parameters: {'kernel': 'rbf', 'C': 20, 'degree': 8, 'gamma': 'scale'}. Best is trial 1 with valu

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'kernel': 'rbf', 'C': 83, 'degree': 9, 'gamma': 'scale'}, {'kernel': 'rbf', 'C': 20, 'degree': 8, 'gamma': 'scale'}, {'kernel': 'rbf', 'C': 92, 'degree': 15, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,1252.574649,0.629912,1090.805934,0.138726,1219.236007,0.723164,936.742845,0.129327
1,0.9,1802.531905,0.233585,1609.389405,0.219560,1982.801860,0.267841,1724.308057,0.245840
2,1.0,1186.652764,0.667842,1024.782143,0.129490,1136.765953,0.759348,863.138905,0.119354


# NIR målinger ekskludert

In [46]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [47]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [48]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:22:21,301] A new study created in memory with name: no-name-02f3a405-c19b-4e71-8dde-a72fc91754c2
[I 2024-03-21 13:22:22,136] Trial 0 finished with value: 1702.246977590236 and parameters: {'n_estimators': 121, 'max_depth': 1}. Best is trial 0 with value: 1702.246977590236.
[I 2024-03-21 13:22:23,110] Trial 1 finished with value: 887.8145148984231 and parameters: {'n_estimators': 110, 'max_depth': 13}. Best is trial 1 with value: 887.8145148984231.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 110, 'max_depth': 13}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,928.825421,0.846876,617.149392,0.088188,322.910672,0.981905,220.395012,0.03278


### KNeighborRegressor

In [49]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:22:23,429] A new study created in memory with name: no-name-939df8f6-85d6-46cb-b3d0-f5fc2293c694
[I 2024-03-21 13:22:23,506] Trial 0 finished with value: 1103.2080519901583 and parameters: {'n_neighbors': 1, 'p': 10}. Best is trial 0 with value: 1103.2080519901583.
[I 2024-03-21 13:22:23,587] Trial 1 finished with value: 990.5908992620218 and parameters: {'n_neighbors': 10, 'p': 6}. Best is trial 1 with value: 990.5908992620218.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 10, 'p': 6}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,908.193522,0.853603,677.188552,0.098546,816.815255,0.884215,619.653824,0.091204


### SupportVectorRegressor

In [50]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:22:23,673] A new study created in memory with name: no-name-21710ad7-55a3-4182-b434-32cb4e0217d6
[I 2024-03-21 13:22:23,734] Trial 0 finished with value: 2313.6718134840667 and parameters: {'kernel': 'poly', 'C': 91, 'degree': 7, 'gamma': 'auto'}. Best is trial 0 with value: 2313.6718134840667.
[I 2024-03-21 13:22:23,814] Trial 1 finished with value: 1462.141498016843 and parameters: {'kernel': 'rbf', 'C': 97, 'degree': 5, 'gamma': 'auto'}. Best is trial 1 with value: 1462.141498016843.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'rbf', 'C': 97, 'degree': 5, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1221.883286,0.735007,891.675945,0.131318,1162.657633,0.765411,902.073888,0.131464


## Alternativ 2: Flere treningssandeler

In [51]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [52]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:22:24,204] A new study created in memory with name: no-name-17ccd0f7-b993-4fb3-9b85-bc78e1cb68a7
[I 2024-03-21 13:22:26,422] Trial 0 finished with value: 1024.3608378063373 and parameters: {'n_estimators': 138, 'max_depth': 20}. Best is trial 0 with value: 1024.3608378063373.
[I 2024-03-21 13:22:28,657] Trial 1 finished with value: 1241.050913114445 and parameters: {'n_estimators': 182, 'max_depth': 3}. Best is trial 0 with value: 1024.3608378063373.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 138, 'max_depth': 20}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,679.319125,0.910312,530.833926,0.074957,309.187018,0.983653,217.271267,0.032429


### KNeighborRegressor

In [53]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:22:29,065] A new study created in memory with name: no-name-f02c97c3-7742-4920-b553-d5ff9c565d7f
[I 2024-03-21 13:22:31,582] Trial 0 finished with value: 1235.5904412027248 and parameters: {'n_neighbors': 5, 'p': 3}. Best is trial 0 with value: 1235.5904412027248.
[I 2024-03-21 13:22:33,803] Trial 1 finished with value: 1153.4465446087775 and parameters: {'n_neighbors': 6, 'p': 19}. Best is trial 1 with value: 1153.4465446087775.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 6, 'p': 19}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,672.898676,0.911999,504.248478,0.070438,766.717832,0.899474,559.044033,0.082805


### SupportVectorRegressor

In [54]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:22:33,888] A new study created in memory with name: no-name-de4e6f85-e104-4392-b1ab-55ff4f04e215
[I 2024-03-21 13:22:35,600] Trial 0 finished with value: 1135.4107582174029 and parameters: {'kernel': 'linear', 'C': 39, 'degree': 10, 'gamma': 'scale'}. Best is trial 0 with value: 1135.4107582174029.
[I 2024-03-21 13:22:37,327] Trial 1 finished with value: 1266.962568418656 and parameters: {'kernel': 'rbf', 'C': 76, 'degree': 18, 'gamma': 'auto'}. Best is trial 0 with value: 1135.4107582174029.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'linear', 'C': 39, 'degree': 10, 'gamma': 'scale'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,758.027771,0.888324,576.893846,0.08174,1099.319567,0.79334,816.851629,0.121681


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [55]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_design_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [56]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:22:37,451] A new study created in memory with name: no-name-dc5bc0e7-f163-4377-8939-825652f8a489
[I 2024-03-21 13:22:38,528] Trial 0 finished with value: 873.6064258760612 and parameters: {'n_estimators': 177, 'max_depth': 7}. Best is trial 0 with value: 873.6064258760612.
[I 2024-03-21 13:22:39,833] Trial 1 finished with value: 873.4092353229885 and parameters: {'n_estimators': 200, 'max_depth': 11}. Best is trial 1 with value: 873.4092353229885.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 200, 'max_depth': 11}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,654.678974,0.910646,468.330937,0.06274,307.237748,0.981987,219.39614,0.029009


### KNeighborRegressor

In [57]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:22:40,287] A new study created in memory with name: no-name-1db2c7d1-1e85-4b38-b73b-c631ce184084
[I 2024-03-21 13:22:40,347] Trial 0 finished with value: 1074.0345033455799 and parameters: {'n_neighbors': 4, 'p': 6}. Best is trial 0 with value: 1074.0345033455799.
[I 2024-03-21 13:22:40,413] Trial 1 finished with value: 1049.2028406390589 and parameters: {'n_neighbors': 6, 'p': 3}. Best is trial 1 with value: 1049.2028406390589.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 6, 'p': 3}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,792.768616,0.868977,608.497,0.085717,776.469517,0.884949,569.810242,0.07716


### SupportVectorRegressor

In [58]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:22:40,485] A new study created in memory with name: no-name-327733d1-cee8-433b-adba-81556c04820c
[I 2024-03-21 13:22:40,539] Trial 0 finished with value: 1126.0150581457526 and parameters: {'kernel': 'linear', 'C': 92, 'degree': 12, 'gamma': 'scale'}. Best is trial 0 with value: 1126.0150581457526.
[I 2024-03-21 13:22:40,596] Trial 1 finished with value: 1143.3740782823731 and parameters: {'kernel': 'linear', 'C': 34, 'degree': 18, 'gamma': 'scale'}. Best is trial 0 with value: 1126.0150581457526.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'linear', 'C': 92, 'degree': 12, 'gamma': 'scale'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1134.824689,0.731519,779.43597,0.115715,1108.314992,0.765595,776.086334,0.108777


## Alternativ 2: Flere treningssandeler

In [59]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [60]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:22:40,815] A new study created in memory with name: no-name-8c1ff8e7-f72e-4180-ae47-2d95ac0d9c73
[I 2024-03-21 13:22:42,856] Trial 0 finished with value: 1132.5958533941312 and parameters: {'n_estimators': 139, 'max_depth': 15}. Best is trial 0 with value: 1132.5958533941312.
[I 2024-03-21 13:22:45,938] Trial 1 finished with value: 1130.2531289388053 and parameters: {'n_estimators': 184, 'max_depth': 10}. Best is trial 1 with value: 1130.2531289388053.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 184, 'max_depth': 10}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,612.203553,0.911592,476.345878,0.061371,324.988079,0.980331,230.125292,0.030649


### KNeighborRegressor

In [61]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:22:46,637] A new study created in memory with name: no-name-eeec7788-13f0-4a37-bd27-955028a21819
[I 2024-03-21 13:22:48,401] Trial 0 finished with value: 1391.7536768444538 and parameters: {'n_neighbors': 11, 'p': 20}. Best is trial 0 with value: 1391.7536768444538.
[I 2024-03-21 13:22:49,887] Trial 1 finished with value: 1357.9038210173858 and parameters: {'n_neighbors': 17, 'p': 20}. Best is trial 1 with value: 1357.9038210173858.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 17, 'p': 20}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,871.423637,0.820875,679.366446,0.086947,1050.327395,0.794555,826.005163,0.112909


### SupportVectorRegressor

In [62]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:22:49,966] A new study created in memory with name: no-name-fb24f20c-76bf-45da-bdae-ce34abb76b9b
[I 2024-03-21 13:22:51,519] Trial 0 finished with value: 2340.087235538107 and parameters: {'kernel': 'poly', 'C': 2, 'degree': 19, 'gamma': 'scale'}. Best is trial 0 with value: 2340.087235538107.
[I 2024-03-21 13:22:52,993] Trial 1 finished with value: 2159.880904938157 and parameters: {'kernel': 'poly', 'C': 16, 'degree': 13, 'gamma': 'auto'}. Best is trial 1 with value: 2159.880904938157.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'poly', 'C': 16, 'degree': 13, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1733.931331,0.290812,1400.048954,0.203313,2097.496682,0.180688,1697.661848,0.254601
